In [ ]:
import re

import pandas as pd
from pathlib import Path
from ploting import *

In [ ]:
root_path = "horeka"

In [ ]:
name_re = re.compile("(weak|strong)_scaling-d_(\d+)-n_(\d+)-([\w_]+)_comm-(gpu|cpu)-nodes_(\d+)")

In [ ]:
time_re = re.compile("DURATION:\s*(\d+(\.\d+)?(e[+-]\d+)?)")

In [ ]:
def create_df(path):
    data = []
    for file in Path(f"{path}/data").iterdir():
        if m := name_re.search(file.name):
            with open(file, "r") as input:
                if t_m := time_re.search(input.read()):
                    duration = t_m.group(1)
                    data.append({"scaling": m.group(1), "dim": m.group(2), "n": m.group(3), "comm_pattern": m.group(4),
                                 "type": m.group(5), "nodes": m.group(6), "time": duration})
    df = pd.DataFrame(data)
    df = df[df.comm_pattern == "optimal"]
    return df.drop(columns=["scaling", "type", "comm_pattern"]).astype({"dim": int, "n": int, "nodes": int, "time": float})

In [ ]:
def pivot(df):
    piv = df.pivot(index="nodes", columns=["dim", "n"], values="time")
    return piv[piv.columns.sort_values()]

In [ ]:
df_with_guard = create_df("crusher-guard")
df_with_guard = pivot(df_with_guard)
df_with_guard

In [ ]:
df_without_guard = create_df("crusher")
df_without_guard = pivot(df_without_guard)
df_without_guard

In [ ]:
df = df_with_guard / df_without_guard
for dim in df_without_guard.columns.levels[0]:
    fig, ax = plt.subplots(dpi=300)
    _df = df[dim]
    ax.set_prop_cycle(default_cycler)
    ax.set_title(f"{dim}D")
    _df.plot(ax=ax, legend=False, logx=True, logy=True)
    ax.legend([f"{n:g}" for n in df.columns.levels[1]])
    #fig.savefig(f"runtime-mpi-comparison-{dim}d.png")